In [1]:
import eunomia
from eunomia.agents import Eunomia
from eunomia.tools import EunomiaTools

In [2]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)
import warnings

# Disable all warning messages
warnings.filterwarnings("ignore")

In [3]:
paper_file =f'95.pdf'
docs_processor = eunomia.LoadDoc(file_name=paper_file, encoding="utf8")
sliced_pages = docs_processor.process([], chunk_size=1000, chunk_overlap=20, chunking_type='fixed-size')

In [6]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

Embedding_model = 'text-embedding-ada-002' 
faiss_index = FAISS.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))

In [21]:
Rule = f"""
There are only 3 options for water stability: 
1. Stable: No change in properties after exposure to moisture or steam, soaking or boiling in water or an aqueous solution.
Retaining its porous structure in solution.
No loss of crystallinity.
Insoluble in water or an aqueous solution.
Water adsorption isotherm should have a steep uptake.
Good cycling performance.

2. Unstable 
The MOF will decompose or change properties or has a change in its crystal structure after exposure/soak to a humid environment, steam or if it partially dissolves in water.
Soluble or patrially soluble in water or an aqueous solution.


3. Not provided, 
If you don't know or cannot justfy 1 or 2
"""

prompt3a = f"""
    You are an expert chemist. The document describes the water stability properties of few metal-organic frameworks.
    Find the exact full name or chemical formula for this material, and its stability in water.
    Find any information on why this MOF is water stable or not from the synthesis steps.
    Find any additional guidlines to design more water-stable MOFs.
    

    Use the following rules to determine water stability:
    {Rule}
    
    Your final answer should contain the following:
    1. The water stability of each MOF.
    2. The exact sentences without any changes from the document that justifies your decision. Try to find more than once sentence. This should be "Not provided" if you cannot find water stability.
    """

In [22]:
from langchain.agents import Tool, tool
from langchain.llms import OpenAI

@tool
def read_doc(input):
    """
    Input the users original prompt and get context on water stability of metal organic frameworks.
    Always search for the answers using this tool first, don't make up answers yourself.
    """

    k = 9
    min_k = 2  # Minimum limit for k
    llm = OpenAI(temperature=0, model_name="gpt-4")
    result = eunomia.RetrievalQABypassTokenLimit(
        prompt3a,
        faiss_index,
        k=k,
        min_k=min_k,
        llm=llm,
        search_type="mmr",
        fetch_k=50,
        chain_type="stuff",
        memory=None,
    )
    return result


tools = [read_doc]

In [23]:
agent = Eunomia(tools=tools, model='gpt-4', get_cost=True)

In [24]:
agent.run(prompt="Read the document and identify detailed guidlines with examples to design more water-stable metal-organic frameworks.")



> Entering new AgentExecutor chain...
I need to read the document to find information on designing more water-stable metal-organic frameworks.
Action: read_doc
Action Input: Detailed guidelines with examples to design more water-stable metal-organic frameworks.
Observation: 1. The water stability of the MOF, barium tetraethyl-1,3,6,8-pyrenetetraphosphonate(CALF-25), is stable. 

2. The following sentences from the document justify this decision: 
- "The ethyl ester groups within the pores also protect CALF-25 from decomposition by water vapor, with crystallinity and porosity being retained after exposure to harsh humid conditions (90% relative humidity at 353 K)."
- "The retention of crystallinity and porosity of CALF-25 at 273 K after exposure to humid conditions suggests that the material is stable toward water vapor."
- "While CALF-25 is stable toward water vapor at 353 K, boiling the MOF in water caused a significant loss in crystallinity (Figure S9), a partial phase change, and 

'The document provides an example of a water-stable MOF, CALF-25, which retains its crystallinity and porosity even after exposure to harsh humid conditions. This is attributed to the ethyl ester groups within the pores that protect the MOF from decomposition by water vapor. The document suggests that the use of phosphonate esters as linkers for the construction of MOFs can protect hydrolytically susceptible coordination backbones. It also suggests that appending organic tethers can enhance the water stability of MOFs. The stability arises from the steric protection offered by ethyl ester groups shielding the inorganic backbone of CALF-25 and defining the nature of the pore.'